In [6]:
import multiprocessing
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import mnist
import time


In [7]:

# Define the data preprocessing function
def preprocess_data(x):
    # Perform some data preprocessing operations
    x = x.reshape(-1, 784).astype('float32') / 255.0
    return x


In [8]:

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [9]:

# Define the number of processes to use
num_processes = multiprocessing.cpu_count()


In [10]:

# Create a multiprocessing Pool
pool = multiprocessing.Pool(processes=num_processes)


In [11]:

# Split the training data into chunks for parallel processing
chunk_size = len(x_train) // num_processes
train_data_chunks = [x_train[i:i+chunk_size] for i in range(0, len(x_train), chunk_size)]


In [12]:

# Start time for data preprocessing
start_time = time.time()


In [13]:

# Apply the data preprocessing function in parallel for training data
preprocessed_train_data = np.concatenate(pool.map(preprocess_data, train_data_chunks))


In [14]:

# End time for data preprocessing
end_time = time.time()
preprocessing_time = end_time - start_time


In [15]:

# Split the test data into chunks for parallel processing
chunk_size = len(x_test) // num_processes
test_data_chunks = [x_test[i:i+chunk_size] for i in range(0, len(x_test), chunk_size)]


In [16]:

# Start time for data preprocessing of test data
start_time = time.time()


In [17]:

# Apply the data preprocessing function in parallel for test data
preprocessed_test_data = np.concatenate(pool.map(preprocess_data, test_data_chunks))


In [18]:

# End time for data preprocessing of test data
end_time = time.time()
preprocessing_time += end_time - start_time


In [19]:

# Close the multiprocessing Pool
pool.close()
pool.join()

print(f"Time taken for data preprocessing: {preprocessing_time:.2f} seconds")


Time taken for data preprocessing: 11.64 seconds


In [20]:

# Use the preprocessed data in your DL task
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(784,)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [21]:

# Start time for model training
start_time = time.time()


In [22]:

# Train the model with the preprocessed data
model.fit(preprocessed_train_data, y_train, epochs=10, batch_size=32, validation_data=(preprocessed_test_data, y_test))


Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2703 - accuracy: 0.9212 - val_loss: 0.1494 - val_accuracy: 0.9550
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1234 - accuracy: 0.9630 - val_loss: 0.1075 - val_accuracy: 0.9687
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0908 - accuracy: 0.9719 - val_loss: 0.1004 - val_accuracy: 0.9697
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0724 - accuracy: 0.9774 - val_loss: 0.1051 - val_accuracy: 0.9663
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0588 - accuracy: 0.9816 - val_loss: 0.0834 - val_accuracy: 0.9750
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0498 - accuracy: 0.9840 - val_loss: 0.0879 - val_accuracy: 0.9757
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0423 - accuracy: 0.9864 - val_loss: 0.1073 - val_accuracy:

In [ ]:

# End time for model training
end_time = time.time()
training_time = end_time - start_time

print(f"Time taken for model training: {training_time:.2f} seconds")


In [5]:

# Evaluate the trained model
start_time = time.time()
_, accuracy = model.evaluate(preprocessed_test_data, y_test)
end_time = time.time()
evaluation_time = end_time - start_time

print(f"Time taken for model evaluation: {evaluation_time:.2f} seconds")
print("Accuracy:", accuracy)


313/313 [==============================] - 1s 2ms/step - loss: 0.1043 - accuracy: 0.9727
Time taken for model evaluation: 0.87 seconds
Accuracy: 0.9726999998092651
